In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('XXX-config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
window_size = int(config.get('Configuration', 'window_size'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))


rooms=config.get('Configuration', 'rooms')
rooms = [(item.strip()) for item in rooms.split(',')]

In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


print(ini_date)
t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))




def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

#day=list(range(day_time(t0),day_time(tN)+1))[0]
day=19798
print(day)

1980-01-01 14:51:20 End date is 315582680 day: 3652 1980-01-01 14:51:20
1980-01-01 13:00:02
1980-01-01 13:00:02 Init date is 315576002 day: 3652 1980-01-01 13:00:02
19798


In [3]:
def getLoc(room,s,off):
    print("S:",s)
    xys=config.get(room, s)
    xys = [(item.strip()) for item in xys.split(',')]
    xys = [list(map(int, item.split('x'))) for item in xys]
    for xy in xys:
        print("\t",xy)
        xy[0]=xy[0]+off[0]
        xy[1]=xy[1]+off[1]
        print("\t+",xy)
    return xys

loc_room_sensor={}
fp_sensors={}
for room in rooms:
    print(room)
    loc_room_sensor[room]={}
    fp_sensors[room] = config.get(room, 'maps')
    fp_sensors[room]  = [(item.strip()) for item in fp_sensors[room].split(',')]

    offW = int(config.get(room, 'offW'))
    offH = int(config.get(room, 'offH'))

    for fp in fp_sensors[room]:
        print(fp)
        loc_room_sensor[room][fp]=getLoc(room,fp,(offH,offW))
        print(loc_room_sensor[room][fp])
print(loc_room_sensor)
print(fp_sensors)

livingroom
s1
S: s1
	 [0, 0]
	+ [0, 2]
	 [0, 1]
	+ [0, 3]
	 [1, 0]
	+ [1, 2]
	 [1, 1]
	+ [1, 3]
[[0, 2], [0, 3], [1, 2], [1, 3]]
s2
S: s2
	 [2, 0]
	+ [2, 2]
	 [2, 1]
	+ [2, 3]
[[2, 2], [2, 3]]
s3
S: s3
	 [3, 0]
	+ [3, 2]
	 [3, 1]
	+ [3, 3]
[[3, 2], [3, 3]]
s4
S: s4
	 [4, 0]
	+ [4, 2]
	 [4, 1]
	+ [4, 3]
[[4, 2], [4, 3]]
s5
S: s5
	 [5, 0]
	+ [5, 2]
	 [5, 1]
	+ [5, 3]
[[5, 2], [5, 3]]
s6
S: s6
	 [6, 0]
	+ [6, 2]
	 [6, 1]
	+ [6, 3]
	 [7, 0]
	+ [7, 2]
	 [7, 1]
	+ [7, 3]
[[6, 2], [6, 3], [7, 2], [7, 3]]
s7
S: s7
	 [0, 2]
	+ [0, 4]
	 [0, 3]
	+ [0, 5]
	 [1, 2]
	+ [1, 4]
	 [1, 3]
	+ [1, 5]
[[0, 4], [0, 5], [1, 4], [1, 5]]
s8
S: s8
	 [2, 2]
	+ [2, 4]
	 [2, 3]
	+ [2, 5]
[[2, 4], [2, 5]]
s9
S: s9
	 [3, 2]
	+ [3, 4]
	 [3, 3]
	+ [3, 5]
[[3, 4], [3, 5]]
s10
S: s10
	 [4, 2]
	+ [4, 4]
	 [4, 3]
	+ [4, 5]
[[4, 4], [4, 5]]
s11
S: s11
	 [5, 3]
	+ [5, 5]
	 [5, 3]
	+ [5, 5]
[[5, 5], [5, 5]]
s12
S: s12
	 [6, 2]
	+ [6, 4]
	 [6, 3]
	+ [6, 5]
	 [7, 2]
	+ [7, 4]
	 [7, 3]
	+ [7, 5]
[[6, 4], [6, 5], [7, 4], [7, 5]]

In [4]:
import os

df_total_map = {}
for room in rooms:
    df_total_map[room]={}
    print(os.listdir(os.path.join(source_folder,room)))
    for fp in fp_sensors[room]:
        df_total_map[room][fp]=None
    for person in os.listdir(os.path.join(source_folder,room)):
        for scene in os.listdir(os.path.join(source_folder,room,person)):
            for fp in fp_sensors[room]:
                print("scene",scene)
                print(os.path.join(source_folder,room,person,scene)+"/binary/"+str(day)+"."+fp+".tsv")
                try:
                    series=read_csv(os.path.join(source_folder,room,person,scene)+"/binary/"+str(day)+"."+fp+".tsv", sep="\t",parse_dates=True, header=None, names=["time","date","state"])
                    print(day,fp,series.size)
                except:
                    print("Not data")
                    continue
                if(df_total_map[room][fp] is None):
                    df_total_map[room][fp]=series
                else:
                    df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)


for room in rooms:
    for fp in fp_sensors[room]:
        if(df_total_map[room][fp] is not None):
            df_total_map[room][fp] = df_total_map[room][fp].sort_values(by='time', ascending=True)
        print(df_total_map[room][fp])


['1-person-labelled', '2-person-labelled', '3-person-labelled']
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/binary/19798.s1.tsv
19798 s1 6
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/binary/19798.s2.tsv
19798 s2 12
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/binary/19798.s3.tsv
19798 s3 12
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/binary/19798.s4.tsv
19798 s4 6
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/binary/19798.s5.tsv
19798 s5 18
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/binary/19798.s6.tsv
19798 s6 6
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/binary/19798.s7.tsv
19798 s7 0
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/binary/19798.s8.tsv
19798 s8 6
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/binary/19798.s9.tsv
19798 s9 12
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/bin

19798 s16 0
scene Scene6
./data/-aligned/livingroom\1-person-labelled\Scene6/binary/19798.s17.tsv
19798 s17 9
scene Scene6
./data/-aligned/livingroom\1-person-labelled\Scene6/binary/19798.s18.tsv
19798 s18 6
scene Scene6
./data/-aligned/livingroom\1-person-labelled\Scene6/binary/19798.s19.tsv
19798 s19 36
scene Scene6
./data/-aligned/livingroom\1-person-labelled\Scene6/binary/19798.s20.tsv
19798 s20 24
scene Scene6
./data/-aligned/livingroom\1-person-labelled\Scene6/binary/19798.s21.tsv
19798 s21 12
scene Scene6
./data/-aligned/livingroom\1-person-labelled\Scene6/binary/19798.s22.tsv
19798 s22 18
scene Scene6
./data/-aligned/livingroom\1-person-labelled\Scene6/binary/19798.s23.tsv
19798 s23 42
scene Scene6
./data/-aligned/livingroom\1-person-labelled\Scene6/binary/19798.s24.tsv
19798 s24 42
scene Scene6
./data/-aligned/livingroom\1-person-labelled\Scene6/binary/19798.s25.tsv
19798 s25 24
scene Scene6
./data/-aligned/livingroom\1-person-labelled\Scene6/binary/19798.s26.tsv
19798 s26 39


C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from

19798 s16 6
scene Scene1
./data/-aligned/livingroom\2-person-labelled\Scene1/binary/19798.s17.tsv
19798 s17 15
scene Scene1
./data/-aligned/livingroom\2-person-labelled\Scene1/binary/19798.s18.tsv
19798 s18 24
scene Scene1
./data/-aligned/livingroom\2-person-labelled\Scene1/binary/19798.s19.tsv
19798 s19 33
scene Scene1
./data/-aligned/livingroom\2-person-labelled\Scene1/binary/19798.s20.tsv
19798 s20 12
scene Scene1
./data/-aligned/livingroom\2-person-labelled\Scene1/binary/19798.s21.tsv
19798 s21 21
scene Scene1
./data/-aligned/livingroom\2-person-labelled\Scene1/binary/19798.s22.tsv
19798 s22 33
scene Scene1
./data/-aligned/livingroom\2-person-labelled\Scene1/binary/19798.s23.tsv
19798 s23 36
scene Scene1
./data/-aligned/livingroom\2-person-labelled\Scene1/binary/19798.s24.tsv
19798 s24 54
scene Scene1
./data/-aligned/livingroom\2-person-labelled\Scene1/binary/19798.s25.tsv
19798 s25 39
scene Scene1
./data/-aligned/livingroom\2-person-labelled\Scene1/binary/19798.s26.tsv
19798 s26 4

19798 s9 0
scene Scene1
./data/-aligned/livingroom\3-person-labelled\Scene1/binary/19798.s10.tsv


...

C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from

19798 c26 9
scene Scene1
./data/-aligned/kitchen\3-person-labelled\Scene1/binary/19798.c27.tsv
19798 c27 0
scene Scene1
./data/-aligned/kitchen\3-person-labelled\Scene1/binary/19798.c28.tsv
19798 c28 15
scene Scene1
./data/-aligned/kitchen\3-person-labelled\Scene1/binary/19798.c29.tsv
19798 c29 18
scene Scene1
./data/-aligned/kitchen\3-person-labelled\Scene1/binary/19798.c30.tsv
19798 c30 18
scene Scene2
./data/-aligned/kitchen\3-person-labelled\Scene2/binary/19798.c1.tsv
19798 c1 15
scene Scene2
./data/-aligned/kitchen\3-person-labelled\Scene2/binary/19798.c2.tsv
19798 c2 15
scene Scene2
./data/-aligned/kitchen\3-person-labelled\Scene2/binary/19798.c3.tsv
19798 c3 27
scene Scene2
./data/-aligned/kitchen\3-person-labelled\Scene2/binary/19798.c4.tsv
19798 c4 9
scene Scene2
./data/-aligned/kitchen\3-person-labelled\Scene2/binary/19798.c5.tsv
19798 c5 21
scene Scene2
./data/-aligned/kitchen\3-person-labelled\Scene2/binary/19798.c6.tsv
19798 c6 48
scene Scene2
./data/-aligned/kitchen\3-per

C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from

19798 b7 6
scene Scene1
./data/-aligned/bathroom\2-person-labelled\Scene1/binary/19798.b8.tsv
19798 b8 0
scene Scene1
./data/-aligned/bathroom\2-person-labelled\Scene1/binary/19798.b9.tsv
19798 b9 36
scene Scene1
./data/-aligned/bathroom\2-person-labelled\Scene1/binary/19798.b10.tsv
19798 b10 39
scene Scene1
./data/-aligned/bathroom\2-person-labelled\Scene1/binary/19798.b11.tsv
19798 b11 36
scene Scene1
./data/-aligned/bathroom\2-person-labelled\Scene1/binary/19798.b12.tsv
19798 b12 48
scene Scene1
./data/-aligned/bathroom\2-person-labelled\Scene1/binary/19798.b13.tsv
19798 b13 36
scene Scene1
./data/-aligned/bathroom\2-person-labelled\Scene1/binary/19798.b14.tsv
19798 b14 6
scene Scene1
./data/-aligned/bathroom\2-person-labelled\Scene1/binary/19798.b15.tsv
19798 b15 0
scene Scene1
./data/-aligned/bathroom\2-person-labelled\Scene1/binary/19798.b16.tsv
19798 b16 0
scene Scene1
./data/-aligned/bathroom\2-person-labelled\Scene1/binary/19798.b17.tsv
19798 b17 6
scene Scene1
./data/-aligned

C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_45460\1530712571.py:23: FutureWarning: The frame.append method is deprecated and will be removed from

In [5]:
import cv2
import os

def getStateTime(df_total,tt):

    df_intervalo1 = df_total[(df_total['time'] >= (tt))]
    df_intervalo2 = df_total[(df_total['time'] <= (tt))]
    #print("tt",tt,time2str(tt))
    #print("df_intervalo1",df_intervalo1, len(df_intervalo1))
    #print("df_intervalo2",df_intervalo2, time2str(tt),tt,time2str(df_intervalo2.iloc[-1]['time']),df_intervalo2.iloc[-1]['time'])
    if(len(df_intervalo2)==0):
        return 100,0
    if(df_intervalo2.iloc[-1]['state']=='on'):
        return tt-df_intervalo2.iloc[-1]['time'],1
    if(df_intervalo2.iloc[-1]['state']=='off'):
        return tt-df_intervalo2.iloc[-1]['time'],-1
    return 100,0
    

os.makedirs(data_folder+"/DAY_"+str(day)+"/", exist_ok=True)
print(t0,tN,time_step)
for tt in range(t0,tN,time_step):
    print(tt,day_time(tt),time2str(tt))
    #print(tt-off_zone,day_time(tt-off_zone),time2str(tt-off_zone))
    #if(day_time(tt) != day):
    #    continue


    img0=np.zeros((h, w))
    v0={}
    min_t={}
    for room in rooms:
        for fp in fp_sensors[room]:
            if not fp in v0:
                v0[fp]=0
                min_t[fp]=100
                
        
        
    for room in rooms:
        for fp in fp_sensors[room]:
            df_total=df_total_map[room][fp]  
            inct,v=getStateTime(df_total,tt)
            #v0[fp]=v0[fp]+v
            #19:35:17
            #if(fp=="s2"):
            #    print("\t",room,fp,inct,v)
            if(inct<min_t[fp] and inct<15):
                min_t[fp]=inct
                v0[fp]=v
                
    for room in rooms:
        for fp in fp_sensors[room]:
            for xy in loc_room_sensor[room][fp]:
                #if(fp=="s2"):
                #    print("\t",v0[fp])
                img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(v0[fp]*255)
        
            #df_intervalo = df_total[(df_total['time'] >= (tt-window_size/2)) & (df_total['time'] <= (tt)) & (df_total['state']=='on')]
            #print("DF",df_intervalo)            
            #for index, row in df_total.iterrows():
            #    dif=tt-row['time']+1
            #    print("@TIME",tt,fp,row, row['time'], "dif:", dif)
            #    for xy in loc_sensor[fp]:
            #        img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(255/dif)
            #for ti in range(int(tt-1),tt+1):
                #print(ti)
                #inct,v=getStateTime(df_total,ti)*0.5
                #dif=tt-ti+1
                #print("@TIMA ",tt,v, "difA:", dif)
                #print(fp,"ti")
                #for xy in loc_room_sensor[room][fp]:
                    #print(fp,xy, "->",v)
                #    img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(v*255/dif)

            #for ti in range(tt+1, int(tt+1+window_size/2)):
            #    v=getStateTime(df_total,ti)*0.25
            #    dif=ti-tt+1
                #print("@TIMB ",tt,v, "difB:", dif)
            #    for xy in loc_room_sensor[room][fp]:
            #        img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(v*255/dif)

    img0[img0 < 0] = 0
    img0[img0 > 255] = 255
    cv2.imwrite(data_folder+"/DAY_"+str(day)+"/FP."+str(tt)+".png", img0) 

315576002 315582680 1
315576002 3652 1980-01-01 13:00:02
315576003 3652 1980-01-01 13:00:03
315576004 3652 1980-01-01 13:00:04
315576005 3652 1980-01-01 13:00:05
315576006 3652 1980-01-01 13:00:06
315576007 3652 1980-01-01 13:00:07
315576008 3652 1980-01-01 13:00:08
315576009 3652 1980-01-01 13:00:09
315576010 3652 1980-01-01 13:00:10
315576011 3652 1980-01-01 13:00:11
315576012 3652 1980-01-01 13:00:12
315576013 3652 1980-01-01 13:00:13
315576014 3652 1980-01-01 13:00:14
315576015 3652 1980-01-01 13:00:15
315576016 3652 1980-01-01 13:00:16
315576017 3652 1980-01-01 13:00:17
315576018 3652 1980-01-01 13:00:18
315576019 3652 1980-01-01 13:00:19
315576020 3652 1980-01-01 13:00:20
315576021 3652 1980-01-01 13:00:21
315576022 3652 1980-01-01 13:00:22
315576023 3652 1980-01-01 13:00:23
315576024 3652 1980-01-01 13:00:24
315576025 3652 1980-01-01 13:00:25
315576026 3652 1980-01-01 13:00:26
315576027 3652 1980-01-01 13:00:27
315576028 3652 1980-01-01 13:00:28
315576029 3652 1980-01-01 13:00:2

315576238 3652 1980-01-01 13:03:58
315576239 3652 1980-01-01 13:03:59
315576240 3652 1980-01-01 13:04:00
315576241 3652 1980-01-01 13:04:01
315576242 3652 1980-01-01 13:04:02
315576243 3652 1980-01-01 13:04:03
315576244 3652 1980-01-01 13:04:04
315576245 3652 1980-01-01 13:04:05
315576246 3652 1980-01-01 13:04:06
315576247 3652 1980-01-01 13:04:07
315576248 3652 1980-01-01 13:04:08
315576249 3652 1980-01-01 13:04:09
315576250 3652 1980-01-01 13:04:10
315576251 3652 1980-01-01 13:04:11
315576252 3652 1980-01-01 13:04:12
315576253 3652 1980-01-01 13:04:13
315576254 3652 1980-01-01 13:04:14
315576255 3652 1980-01-01 13:04:15
315576256 3652 1980-01-01 13:04:16
315576257 3652 1980-01-01 13:04:17
315576258 3652 1980-01-01 13:04:18
315576259 3652 1980-01-01 13:04:19
315576260 3652 1980-01-01 13:04:20
315576261 3652 1980-01-01 13:04:21
315576262 3652 1980-01-01 13:04:22
315576263 3652 1980-01-01 13:04:23
315576264 3652 1980-01-01 13:04:24
315576265 3652 1980-01-01 13:04:25
315576266 3652 1980-

315576473 3652 1980-01-01 13:07:53
315576474 3652 1980-01-01 13:07:54
315576475 3652 1980-01-01 13:07:55
315576476 3652 1980-01-01 13:07:56
315576477 3652 1980-01-01 13:07:57
315576478 3652 1980-01-01 13:07:58
315576479 3652 1980-01-01 13:07:59
315576480 3652 1980-01-01 13:08:00
315576481 3652 1980-01-01 13:08:01
315576482 3652 1980-01-01 13:08:02
315576483 3652 1980-01-01 13:08:03
315576484 3652 1980-01-01 13:08:04
315576485 3652 1980-01-01 13:08:05
315576486 3652 1980-01-01 13:08:06
315576487 3652 1980-01-01 13:08:07
315576488 3652 1980-01-01 13:08:08
315576489 3652 1980-01-01 13:08:09
315576490 3652 1980-01-01 13:08:10
315576491 3652 1980-01-01 13:08:11
315576492 3652 1980-01-01 13:08:12
315576493 3652 1980-01-01 13:08:13
315576494 3652 1980-01-01 13:08:14
315576495 3652 1980-01-01 13:08:15
315576496 3652 1980-01-01 13:08:16
315576497 3652 1980-01-01 13:08:17
315576498 3652 1980-01-01 13:08:18
315576499 3652 1980-01-01 13:08:19
315576500 3652 1980-01-01 13:08:20
315576501 3652 1980-

315576712 3652 1980-01-01 13:11:52
315576713 3652 1980-01-01 13:11:53
315576714 3652 1980-01-01 13:11:54
315576715 3652 1980-01-01 13:11:55
315576716 3652 1980-01-01 13:11:56
315576717 3652 1980-01-01 13:11:57
315576718 3652 1980-01-01 13:11:58
315576719 3652 1980-01-01 13:11:59
315576720 3652 1980-01-01 13:12:00
315576721 3652 1980-01-01 13:12:01
315576722 3652 1980-01-01 13:12:02
315576723 3652 1980-01-01 13:12:03
315576724 3652 1980-01-01 13:12:04
315576725 3652 1980-01-01 13:12:05
315576726 3652 1980-01-01 13:12:06
315576727 3652 1980-01-01 13:12:07
315576728 3652 1980-01-01 13:12:08
315576729 3652 1980-01-01 13:12:09
315576730 3652 1980-01-01 13:12:10
315576731 3652 1980-01-01 13:12:11
315576732 3652 1980-01-01 13:12:12
315576733 3652 1980-01-01 13:12:13
315576734 3652 1980-01-01 13:12:14
315576735 3652 1980-01-01 13:12:15
315576736 3652 1980-01-01 13:12:16
315576737 3652 1980-01-01 13:12:17
315576738 3652 1980-01-01 13:12:18
315576739 3652 1980-01-01 13:12:19
315576740 3652 1980-

315576951 3652 1980-01-01 13:15:51
315576952 3652 1980-01-01 13:15:52
315576953 3652 1980-01-01 13:15:53
315576954 3652 1980-01-01 13:15:54
315576955 3652 1980-01-01 13:15:55
315576956 3652 1980-01-01 13:15:56
315576957 3652 1980-01-01 13:15:57
315576958 3652 1980-01-01 13:15:58
315576959 3652 1980-01-01 13:15:59
315576960 3652 1980-01-01 13:16:00
315576961 3652 1980-01-01 13:16:01
315576962 3652 1980-01-01 13:16:02
315576963 3652 1980-01-01 13:16:03
315576964 3652 1980-01-01 13:16:04
315576965 3652 1980-01-01 13:16:05
315576966 3652 1980-01-01 13:16:06
315576967 3652 1980-01-01 13:16:07
315576968 3652 1980-01-01 13:16:08
315576969 3652 1980-01-01 13:16:09
315576970 3652 1980-01-01 13:16:10
315576971 3652 1980-01-01 13:16:11
315576972 3652 1980-01-01 13:16:12
315576973 3652 1980-01-01 13:16:13
315576974 3652 1980-01-01 13:16:14
315576975 3652 1980-01-01 13:16:15
315576976 3652 1980-01-01 13:16:16
315576977 3652 1980-01-01 13:16:17
315576978 3652 1980-01-01 13:16:18
315576979 3652 1980-

...

315581680 3652 1980-01-01 14:34:40
315581681 3652 1980-01-01 14:34:41
315581682 3652 1980-01-01 14:34:42
315581683 3652 1980-01-01 14:34:43
315581684 3652 1980-01-01 14:34:44
315581685 3652 1980-01-01 14:34:45
315581686 3652 1980-01-01 14:34:46
315581687 3652 1980-01-01 14:34:47
315581688 3652 1980-01-01 14:34:48
315581689 3652 1980-01-01 14:34:49
315581690 3652 1980-01-01 14:34:50
315581691 3652 1980-01-01 14:34:51
315581692 3652 1980-01-01 14:34:52
315581693 3652 1980-01-01 14:34:53
315581694 3652 1980-01-01 14:34:54
315581695 3652 1980-01-01 14:34:55
315581696 3652 1980-01-01 14:34:56
315581697 3652 1980-01-01 14:34:57
315581698 3652 1980-01-01 14:34:58
315581699 3652 1980-01-01 14:34:59
315581700 3652 1980-01-01 14:35:00
315581701 3652 1980-01-01 14:35:01
315581702 3652 1980-01-01 14:35:02
315581703 3652 1980-01-01 14:35:03
315581704 3652 1980-01-01 14:35:04
315581705 3652 1980-01-01 14:35:05
315581706 3652 1980-01-01 14:35:06
315581707 3652 1980-01-01 14:35:07
315581708 3652 1980-

315581917 3652 1980-01-01 14:38:37
315581918 3652 1980-01-01 14:38:38
315581919 3652 1980-01-01 14:38:39
315581920 3652 1980-01-01 14:38:40
315581921 3652 1980-01-01 14:38:41
315581922 3652 1980-01-01 14:38:42
315581923 3652 1980-01-01 14:38:43
315581924 3652 1980-01-01 14:38:44
315581925 3652 1980-01-01 14:38:45
315581926 3652 1980-01-01 14:38:46
315581927 3652 1980-01-01 14:38:47
315581928 3652 1980-01-01 14:38:48
315581929 3652 1980-01-01 14:38:49
315581930 3652 1980-01-01 14:38:50
315581931 3652 1980-01-01 14:38:51
315581932 3652 1980-01-01 14:38:52
315581933 3652 1980-01-01 14:38:53
315581934 3652 1980-01-01 14:38:54
315581935 3652 1980-01-01 14:38:55
315581936 3652 1980-01-01 14:38:56
315581937 3652 1980-01-01 14:38:57
315581938 3652 1980-01-01 14:38:58
315581939 3652 1980-01-01 14:38:59
315581940 3652 1980-01-01 14:39:00
315581941 3652 1980-01-01 14:39:01
315581942 3652 1980-01-01 14:39:02
315581943 3652 1980-01-01 14:39:03
315581944 3652 1980-01-01 14:39:04
315581945 3652 1980-

315582152 3652 1980-01-01 14:42:32
315582153 3652 1980-01-01 14:42:33
315582154 3652 1980-01-01 14:42:34
315582155 3652 1980-01-01 14:42:35
315582156 3652 1980-01-01 14:42:36
315582157 3652 1980-01-01 14:42:37
315582158 3652 1980-01-01 14:42:38
315582159 3652 1980-01-01 14:42:39
315582160 3652 1980-01-01 14:42:40
315582161 3652 1980-01-01 14:42:41
315582162 3652 1980-01-01 14:42:42
315582163 3652 1980-01-01 14:42:43
315582164 3652 1980-01-01 14:42:44
315582165 3652 1980-01-01 14:42:45
315582166 3652 1980-01-01 14:42:46
315582167 3652 1980-01-01 14:42:47
315582168 3652 1980-01-01 14:42:48
315582169 3652 1980-01-01 14:42:49
315582170 3652 1980-01-01 14:42:50
315582171 3652 1980-01-01 14:42:51
315582172 3652 1980-01-01 14:42:52
315582173 3652 1980-01-01 14:42:53
315582174 3652 1980-01-01 14:42:54
315582175 3652 1980-01-01 14:42:55
315582176 3652 1980-01-01 14:42:56
315582177 3652 1980-01-01 14:42:57
315582178 3652 1980-01-01 14:42:58
315582179 3652 1980-01-01 14:42:59
315582180 3652 1980-

315582389 3652 1980-01-01 14:46:29
315582390 3652 1980-01-01 14:46:30
315582391 3652 1980-01-01 14:46:31
315582392 3652 1980-01-01 14:46:32
315582393 3652 1980-01-01 14:46:33
315582394 3652 1980-01-01 14:46:34
315582395 3652 1980-01-01 14:46:35
315582396 3652 1980-01-01 14:46:36
315582397 3652 1980-01-01 14:46:37
315582398 3652 1980-01-01 14:46:38
315582399 3652 1980-01-01 14:46:39
315582400 3652 1980-01-01 14:46:40
315582401 3652 1980-01-01 14:46:41
315582402 3652 1980-01-01 14:46:42
315582403 3652 1980-01-01 14:46:43
315582404 3652 1980-01-01 14:46:44
315582405 3652 1980-01-01 14:46:45
315582406 3652 1980-01-01 14:46:46
315582407 3652 1980-01-01 14:46:47
315582408 3652 1980-01-01 14:46:48
315582409 3652 1980-01-01 14:46:49
315582410 3652 1980-01-01 14:46:50
315582411 3652 1980-01-01 14:46:51
315582412 3652 1980-01-01 14:46:52
315582413 3652 1980-01-01 14:46:53
315582414 3652 1980-01-01 14:46:54
315582415 3652 1980-01-01 14:46:55
315582416 3652 1980-01-01 14:46:56
315582417 3652 1980-

315582624 3652 1980-01-01 14:50:24
315582625 3652 1980-01-01 14:50:25
315582626 3652 1980-01-01 14:50:26
315582627 3652 1980-01-01 14:50:27
315582628 3652 1980-01-01 14:50:28
315582629 3652 1980-01-01 14:50:29
315582630 3652 1980-01-01 14:50:30
315582631 3652 1980-01-01 14:50:31
315582632 3652 1980-01-01 14:50:32
315582633 3652 1980-01-01 14:50:33
315582634 3652 1980-01-01 14:50:34
315582635 3652 1980-01-01 14:50:35
315582636 3652 1980-01-01 14:50:36
315582637 3652 1980-01-01 14:50:37
315582638 3652 1980-01-01 14:50:38
315582639 3652 1980-01-01 14:50:39
315582640 3652 1980-01-01 14:50:40
315582641 3652 1980-01-01 14:50:41
315582642 3652 1980-01-01 14:50:42
315582643 3652 1980-01-01 14:50:43
315582644 3652 1980-01-01 14:50:44
315582645 3652 1980-01-01 14:50:45
315582646 3652 1980-01-01 14:50:46
315582647 3652 1980-01-01 14:50:47
315582648 3652 1980-01-01 14:50:48
315582649 3652 1980-01-01 14:50:49
315582650 3652 1980-01-01 14:50:50
315582651 3652 1980-01-01 14:50:51
315582652 3652 1980-